<a href="https://colab.research.google.com/github/Nnamaka/ML_specializations/blob/main/Computer_vision/face_Detection/faceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir dataset && tar zxvf dataimages.tgz -C dataset

In [ ]:
!rm -r dataset

In [ ]:
!rm dataimages.tgz

In [ ]:
!wget -O save.jpg https://image.shutterstock.com/image-photo/smart-robotic-agriculture-futuristic-concept-260nw-1314154001.jpg

#Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib
!pip install wget

##Import Dependencies

In [ ]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

##Create Folder Structures


In [ ]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [ ]:
# make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

#Collect Positive and Anchors
My positive images were stored in my Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!cp "/content/drive/MyDrive/TFOD images/face.tar.gz" /content

uncompressed data file of positive image

In [ ]:
!tar -xzf face.tar.gz

resize and store images as positive images

In [ ]:
width = 250
height = 250
dim = (width, height)

# TODO: get the proper directory to search for images
for filename in os.listdir(""):
  if filename.endswith('.jpg'):
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    new_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    # TODO: make names unique with uuid()
    cv2.imwrite("...", new_img)

In [ ]:
# TODO: snap and collect anchor images

#Collect Negative Images

In [ ]:
!wget -O wildImages.tar http://vis-www.cs.umass.edu/lfw/lfw.tgz

--2022-09-18 12:31:20--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘wildImages.tar’

wildImages.tar      100%[===================>] 172.20M  15.2MB/s    in 13s     

2022-09-18 12:31:33 (13.5 MB/s) - ‘wildImages.tar’ saved [180566744/180566744]



###Untar Labelled Faces in the wild Dataset



In [ ]:
!tar -xzf wildImages.tar

In [ ]:
# move ifw images to the negative folder image directory

for directory in os.listdir('lfw'):
  for file in os.listdir(os.path.join('lfw', directory)):
    old_path = os.path.join('lfw', directory, file)
    new_path = os.path.join(NEG_PATH, file)
    os.replace(old_path, new_path)